# Building a Libor Curve

This is an example of a replication of a Quantlib example from

http://billiontrader.com/2015/02/16/bootstrapping-with-quantlib/

Agreement is very good however some issues about date generation need to be checked.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [1]:
from financepy.finutils import *
from financepy.products.libor import *

###################################################################
# FINANCEPY BETA Version 0.180 - This build: 22 Sep 2020 at 19:38 #
#     This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy #
###################################################################



## Creating the Libor Instruments

In [2]:
tradeDate = FinDate(16, 2, 2015)

### Load up the deposits first

In [3]:
spotDays = 0
settlementDate = tradeDate.addWorkDays(spotDays)

In [4]:
depoDCCType = FinDayCountTypes.ACT_360
depos = []

depositRate = 0.001375
depo = FinLiborDeposit(settlementDate, "7D", depositRate, depoDCCType)
depos.append(depo)

depositRate = 0.001717
depo = FinLiborDeposit(settlementDate, "1M", depositRate, depoDCCType)
depos.append(depo)

depositRate = 0.002112
depo = FinLiborDeposit(settlementDate, "2M", depositRate, depoDCCType)
depos.append(depo)

depositRate = 0.002581
depo = FinLiborDeposit(settlementDate, "3M", depositRate, depoDCCType)
depos.append(depo)

In [5]:
for depo in depos:
    print(depo)

OBJECT TYPE: FinLiborDeposit
START DATE: MON 16 FEB 2015
MATURITY DATE: MON 23 FEB 2015
NOTIONAL: 100.0
DEPOSIT RATE: 0.001375
DAY COUNT TYPE: FinDayCountTypes.ACT_360
CALENDAR: FinCalendarTypes.WEEKEND
BUS DAY ADJUST TYPE: FinBusDayAdjustTypes.MODIFIED_FOLLOWING

OBJECT TYPE: FinLiborDeposit
START DATE: MON 16 FEB 2015
MATURITY DATE: MON 16 MAR 2015
NOTIONAL: 100.0
DEPOSIT RATE: 0.001717
DAY COUNT TYPE: FinDayCountTypes.ACT_360
CALENDAR: FinCalendarTypes.WEEKEND
BUS DAY ADJUST TYPE: FinBusDayAdjustTypes.MODIFIED_FOLLOWING

OBJECT TYPE: FinLiborDeposit
START DATE: MON 16 FEB 2015
MATURITY DATE: THU 16 APR 2015
NOTIONAL: 100.0
DEPOSIT RATE: 0.002112
DAY COUNT TYPE: FinDayCountTypes.ACT_360
CALENDAR: FinCalendarTypes.WEEKEND
BUS DAY ADJUST TYPE: FinBusDayAdjustTypes.MODIFIED_FOLLOWING

OBJECT TYPE: FinLiborDeposit
START DATE: MON 16 FEB 2015
MATURITY DATE: MON 18 MAY 2015
NOTIONAL: 100.0
DEPOSIT RATE: 0.002581
DAY COUNT TYPE: FinDayCountTypes.ACT_360
CALENDAR: FinCalendarTypes.WEEKEND
BU

### Create Strips of Interest Rate Futures 

In [6]:
futs = []
fut = FinLiborFuture(tradeDate, 1) ; futs.append(fut)
fut = FinLiborFuture(tradeDate, 2) ; futs.append(fut)
fut = FinLiborFuture(tradeDate, 3) ; futs.append(fut)
fut = FinLiborFuture(tradeDate, 4) ; futs.append(fut)
fut = FinLiborFuture(tradeDate, 5) ; futs.append(fut)
fut = FinLiborFuture(tradeDate, 6) ; futs.append(fut)

### Convert Interest Rate Futures to FRAs

Need to supply futures price and the convexity (in percent) which is set to zero

In [7]:
fras = [None]*len(futs)
fras[0] = futs[0].toFRA(99.725,-0.0)
fras[1] = futs[1].toFRA(99.585,-0.00)
fras[2] = futs[2].toFRA(99.385,-0.00)
fras[3] = futs[3].toFRA(99.160,-0.00)
fras[4] = futs[4].toFRA(98.930,-0.00)
fras[5] = futs[5].toFRA(98.715,-0.00)

### Then we load up swap rates

In [11]:
accrual = FinDayCountTypes.ACT_360
freq = FinFrequencyTypes.ANNUAL
longEnd = FinDateGenRuleTypes.BACKWARD
swapType = FinLiborSwapTypes.PAYER

In [9]:
spotDays = 2
settlementDate = tradeDate.addWorkDays(spotDays)

In [12]:
swaps = []
swap = FinLiborSwap(settlementDate, "2Y", swapType, 0.0089268, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "3Y", swapType, 0.0123343, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "4Y", swapType, 0.0147985, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "5Y", swapType, 0.0165843, freq, accrual); swaps.append(swap)

In [13]:
for swap in swaps:
    print(swap._adjustedMaturityDate)

MON 20 FEB 2017
MON 19 FEB 2018
MON 18 FEB 2019
TUE 18 FEB 2020


## Build the Curve

In [13]:
liborCurve = FinLiborCurve(tradeDate, depos, fras, swaps)

## Curve Examination

Generate a vector of time points and then generate zero rates and forward rates

In [14]:
for depo in depos:
    dt = depo._maturityDate
    df = liborCurve.df(dt)
    zeroRate = liborCurve.zeroRate(dt, FinFrequencyTypes.SIMPLE, FinDayCountTypes.ACT_360)
    df = liborCurve.df(dt)
    print("%12s %12.8f %12.8f" % (dt, zeroRate*100.0, df))

MON 23 FEB 2015   0.13750000   0.99997326
MON 16 MAR 2015   0.17170000   0.99986647
THU 16 APR 2015   0.21120000   0.99965399
MON 18 MAY 2015   0.25810000   0.99934801


In [15]:
for fra in fras:
    dt = fra._maturityDate
    df = liborCurve.df(dt)
    zeroRate = liborCurve.zeroRate(dt, FinFrequencyTypes.SIMPLE, FinDayCountTypes.ACT_360)
    print("%12s %12.8f %12.8f" % (dt, zeroRate*100.0, df))

WED 17 JUN 2015   0.25066108   0.99915821
WED 16 SEP 2015   0.32135287   0.99811116
WED 16 DEC 2015   0.40989345   0.99656192
WED 16 MAR 2016   0.50990211   0.99445037
WED 15 JUN 2016   0.61611302   0.99176792
WED 21 SEP 2016   0.73034320   0.98831075


In [16]:
for swap in swaps:
    dt = swap._adjustedMaturityDate
    df = liborCurve.df(dt)
    zeroRate = liborCurve.zeroRate(dt, FinFrequencyTypes.ANNUAL, FinDayCountTypes.ACT_360)
    print("%12s %12.8f %12.8f" % (dt, zeroRate*100.0, df))

MON 20 FEB 2017   0.89237549   0.98202499
MON 19 FEB 2018   1.23733169   0.96315473
MON 18 FEB 2019   1.48896351   0.94170449
TUE 18 FEB 2020   1.67274927   0.91921438


In [17]:
swaps[0].value(settlementDate, liborCurve, liborCurve)

-5.10702591327572e-15

In [18]:
swaps[0].printFixedLegPV()

START DATE: WED 18 FEB 2015
MATURITY DATE: MON 20 FEB 2017
COUPON (%): 0.89268
FIXED LEG FREQUENCY: FinFrequencyTypes.ANNUAL
FIXED LEG DAY COUNT: FinDayCountTypes.ACT_360
VALUATION DATE WED 18 FEB 2015
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
WED 18 FEB 2015          -            -   1.00000000            -            -
THU 18 FEB 2016  1.0138889         0.91   0.99508401         0.90         0.90
MON 20 FEB 2017  1.0222222         0.91   0.98203249         0.90         1.80


In [19]:
swaps[2].printFloatLegPV()

START DATE: WED 18 FEB 2015
MATURITY DATE: MON 18 FEB 2019
SPREAD COUPON (%): 0.0
FLOAT LEG FREQUENCY: FinFrequencyTypes.QUARTERLY
FLOAT LEG DAY COUNT: FinDayCountTypes.THIRTY_E_360
VALUATION DATE None
Floating Flows not calculated.


Agreement with Quantlib example is very very good but not exact. Not sure how some of quantlib dates are generated - e.g. 19/2/2019 is a Tuesday so the 18th was a weekday and should have been used ?

Copyright (c) 2020 Dominic O'Kane